In [ ]:
import pandas as pd

In [ ]:
file = "legacy_data/2018_09_05_SW_Zusammenfuehrung FINAL.xls"
temp_scheme, _ = SkosConceptScheme.objects.get_or_create(
    dc_title='schlagwort'
)

In [ ]:
df = pd.read_excel(file).fillna('False')

In [ ]:
df.head()

In [ ]:
concepts = []
for i, row in df.iterrows():
    if row[0] != 'False':
        related_concepts = []
        for x in range(0, 3):
            pref_labels = []
            alt_labels = []
            pref_labels = row[x].split(';')
            pref_label = pref_labels[0].strip()
            concept, _ = SkosConcept.objects.get_or_create(pref_label=pref_label)
            related_concepts.append(concept)
            concept.scheme.set([temp_scheme])
            if len(pref_labels) > 1:
                for y in pref_labels[1:]:
                    label_name = y.strip()
                    label, _ = SkosLabel.objects.get_or_create(
                        label=y.strip(),
                        label_type='altLabel',
                        isoCode='ger'
                    )
                    alt_labels.append(label)
            if alt_labels:
                concept.other_label.set(alt_labels)
        concepts.append(related_concepts)

In [ ]:
for row in concepts:
    x = 0
    broader = None
    narrower = None
    while x != len(row):
        broader = row[x]
        try:
            narrower = row[x+1]
        except:
            narrower = False
        if narrower:
            narrower.broader_concept = broader
            narrower.save()
        x +=1